In [13]:
import os

In [29]:
%pwd

'd:\\FutureExpertData'

In [31]:
os.chdir("./Text-Summarizer-Project")

In [32]:
%pwd

'd:\\FutureExpertData\\Text-Summarizer-Project'

In [33]:
%pwd

'd:\\FutureExpertData\\Text-Summarizer-Project'

In [34]:
#define the entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig: 
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [35]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [36]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root]) #automatically create the folder that will contain the artefacts
        
        
    def get_data_transformation_config(self) -> DataTransformationConfig: 
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir, 
            data_path= config.data_path, 
            tokenizer_name= config.tokenizer_name
        )
        
        return data_transformation_config 
        

In [37]:
import os 
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [38]:



class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
        
    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 512, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer (example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'], #The tensor of ids of the tokens from the original text
            'attention_mask': input_encodings['attention_mask'], #the tensor that tells the model which token to pay attention to (1 or 0)
            'labels': target_encodings['input_ids']  #The tensor of ids of the tokens from the summary
        }
     
    def convert(self):
        data_samsum = load_from_disk(self.config.data_path)
        data_samsum_pt = data_samsum.map(self.convert_examples_to_features, batched=True)
        data_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))
    

In [39]:
#data creation 
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-07-18 16:18:57,865: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2025-07-18 16:18:57,867: INFO: common: yaml file: params.yaml loaded successfully.]
[2025-07-18 16:18:57,868: INFO: common: Created directory at: artifacts]
[2025-07-18 16:18:57,869: INFO: common: Created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]d:\FutureExpertData\Text-Summarizer-Project\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 164632.47 examples/s]
